In [1]:
#Nozzle Thrust and Altitude Simulation
# Peter Strohmaier 9/1/19
# HALE Rocket Team


import numpy as np
import matplotlib.pyplot as plt
import math

#Conversions
def poundpersec_to_kilogrampersec(i):
    return i * 0.453592
def pound_to_kilogram(i):
    return i * 0.453592
def poundforce_to_newton(i):
    return i * 4.44822
def inch_to_meter(i):
    return i * 0.0254

#Defining Arrays and Start Points
t = 0
Accrocket = 0 #Acceleration of Rocket
AccVec = np.array([])
Vrocket = 0 #Velocity of Rocket, m/s
VelocityVec = np.array([])
Altitude = 0 #Altitude, m
AltitudeVec = np.array([])
Max_Altitudes = np.array([])
Max_Acc = np.array([])
Max_Velocity = np.array([])
dt = 0.01


#Known Values
expansion_ratio = np.linspace(1.0, 40.0, num = 40)
mdot = poundpersec_to_kilogrampersec(7.72) #kg/s
m_full = pound_to_kilogram(150) #kg
Cd = 2
Arocket = math.pi*((inch_to_meter(6)/2)**2) #External Cross-Sectional Area, m^2
g = 9.81 #m/s^2


In [2]:
#Density as a Function of Altitude
def air_density(i):
    altitude = np.array([0,1000,3000,5000,10000,25000,50000,75000,100000,\
                     130000,160000,200000,300000,400000,600000,1000000])
    density = np.array([1.2250,1.1117,9.0912*(10**-1),7.6312*(10**-1),4.1351*(10**-1),\
                   4.0084*(10**-2),1.0269*(10**-3),3.4861*(10**-5),5.604*(10**-7),\
                   8.152*(10**-9),1.233*(10**-9),2.541*(10**-10),1.916*(10**-11),\
                   2.803*(10**-12),2.137*(10**-13),3.561*(10**-15)])
    return np.interp(i,altitude,density) #kg/m^3

#Ambient Pressure as a Function of Altitude
def air_pressure(i):
    altitude = np.array([0,1000,3000,5000,10000,25000,50000,75000,100000,\
                     130000,160000,200000,300000,400000,600000,1000000])
    pressure_ratio = np.array([1,8.8700*(10**-1),6.6919*(10**-1),5.3313*(10**-1),\
                              2.6151*(10**-1),2.5158*(10**-2),7.8735*(10**-4),2.0408*(10**-5),\
                              3.1593*(10**-7),1.2341*(10**-8),2.9997*(10**-9),8.3628*(10**-10),\
                              8.6557*(10**-11),1.4328*(10**-11),8.1056*(10**-13),7.4155*(10**-14)])
    x = np.interp(i,altitude,pressure_ratio)
    return x * (101325) #Pa



In [3]:
#Function for Thrust Force
def Fthrust(i):
    return poundforce_to_newton(i) #N


In [4]:
#Function for Drag Force
def Fdrag(Vrocket,Altitude):
    return (Cd*(Vrocket**2)*air_density(Altitude)*Arocket)/2 #N
    

In [5]:
#Flight Performance Simulation

#Primary Iteration Over Each Expansion Ratio
for exp in np.nditer(expansion_ratio):
    
    #Seconday Iteration for each time step
    for i in range(2001):
        t = t + dt
        
        #Updating the Instantaneous Mass
        if t < 5.5:
            m_PropellantConsumed = mdot*t
            m_inst = m_full - m_PropellantConsumed
        else:
            m_inst = m_full - m_PropellantConsumed
        
        Weight = m_inst*g
        Fnet = Fthrust(2000) - Fdrag(Vrocket,Altitude) - Weight
        
        Accrocket = Fnet/m_inst
        Vrocket = Vrocket + Accrocket*dt
        Altitude = Altitude + Vrocket*dt
        
        if len(AccVec) <= 2000:
            AccVec = np.append(AccVec,Accrocket)
            VelocityVec = np.append(VelocityVec,Vrocket)
            AltitudeVec = np.append(AltitudeVec,Altitude)

        else:
            AccVec = 0
            VelocityVec = 0
            AltitudeVec = 0
        
    #Record Max Altitude Reached at Each Expansion Ratio
    Max_Acc = np.append(Max_Acc,max(AccVec))
    Max_Velocity = np.append(Max_Velocity,max(VelocityVec))
    Max_Altitudes = np.append(Max_Altitudes,max(AltitudeVec))
    

    ######## current Issue (Lines 24-32) #########
    #I need to reset the length for my storage arrays after each complete cycle over 
    #one expansion ratio, however by using a len function to see if the length of stored
    #variables is less than or equal to the iterations through, an error is returned because the 
    #array initially begins with no length.
        

TypeError: object of type 'int' has no len()

In [ ]:
#Plotting Expansion Ratio vs Altitude
plt.plot(expansion_ratio,Max_Altitudes)
